In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from glob import glob
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
main_dir = 'C:/e/CRUPA/MISHA_heap_v2/'
main_dir_Anton = 'C:/e/CRUPA/ANTON_heap/'

# DATA GENERATOR

In [4]:

img_shape = (256, 256)
images_mish = sorted(os.listdir(main_dir))
images_anton = sorted(os.listdir(main_dir_Anton))
#images_misha = os.listdir(main_dir_Mish)
images = images_mish + images_anton 

images_info_df = pd.DataFrame(columns=['image_name', 'mass', 'food_type'])
for ind, img in enumerate(images):
    items = img.split('_')
    file = img.split('_')[2]
    #print(file[0:2])
    if file[0:2] == 'mv':
      images_info_df.loc[ind] = [main_dir + img, int(items[0]), items[1]]
    elif file[0:2] == 'av':
      images_info_df.loc[ind] = [main_dir_Anton + img, int(items[0]), items[1]]
    
#images_info_df['image_name'] = sorted(images_info_df['image_name'])
mass_list = list(images_info_df['mass'])  
mass_list = np.array(mass_list)

In [5]:
images_list = []
images_batch = []
mass_list = []
mass_batch = []
j = 0
for i, file in enumerate(images_info_df['image_name']):
  #print(i,file)
  if j < 5:
    j += 1
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_shape)
    images_batch.append(img)
    mass_batch.append(images_info_df['mass'][i])
    
  else:
    j = 0 
    images_list.append(images_batch)
    mass_list.append(int(images_info_df['mass'][i])) 
    images_batch = [] 
    mass_batch = []

images_list = np.array(images_list)
mass_list = np.array(mass_list)    

print(mass_list)

[100 100 100 100 100 100 100 100 100 100  10  10  10  10  10  10  10  10
  10  10 110 110 110 110 110 110 110 110 110 110 130 130 130 130 130 130
 130 130 130 130 140 140 140 140 140 140 140 140 140 140 150 150 150 150
 150 150 150 150 150 150 160 160 160 160 160 160 160 160 160 160 170 170
 170 170 170 170 170 170 170 170 180 180 180 180 180 180 180 180 180 180
 190 190 190 190 190 190 190 190 190 190 200 200 200 200 200 200 200 200
 200 200  20  20  20  20  20  20  20  20  20  20  30  30  30  30  30  30
  30  30  30  30  40  40  40  40  40  40  40  40  40  40  50  50  50  50
  50  50  50  50  50  50  60  60  60  60  60  60  60  60  60  60  70  70
  70  70  70  70  70  70  70  70  80  80  80  80  80  80  80  80  80  80
  90  90  90  90  90  90  90  90  90  90 100 100  10  10 110 110 120 120
 130 130 140 140 150 150 160 160 170 170 180 180 190 190 200 200  20  30
  30  40  40  50  50  60  60  70  70  80  90  90]


# MODEL

In [7]:
import keras
from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import MobileNet,ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
# create a VGG16 "model", we will use
# image with shape (224, 224, 3)
base_model=ResNet50( weights='imagenet',include_top=False,input_shape=(256,256,3))
# vgg = VGG16(
#     include_top=False,
#     weights='imagenet',
#     input_shape=(224, 224, 3)
# )

for layer in base_model.layers[:-4]:
    layer.trainable = False

model = Sequential()

model.add(
    TimeDistributed(base_model, input_shape=(5, 256, 256, 3))
)

model.add(
    TimeDistributed(
        #Flatten()
        GlobalAveragePooling2D()
    )
)
model.add(LSTM(256, activation='relu', return_sequences=False))

model.add(Dense(64, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='linear'))
model.compile(optimizer = 'adam', loss='mean_absolute_error')

In [8]:
model = keras.models.load_model('C:/e/CRUPA/save/model_TIMEDIST_Resnet.h5')

# PREDICT 

In [10]:
y_pred = []
y_true = []
for i in range(120,200,5):
    
    img_sample = images_list[i]
    img_sample = np.expand_dims(img_sample, axis=0)
    pred = model.predict(img_sample)
    y_pred.append(pred)
    y_true.append(images_info_df['mass'][i])
    print(images_info_df['mass'][i], '-true')
    print(pred, '- predict')

110 -true
[[29.033297]] - predict
110 -true
[[34.17439]] - predict
110 -true
[[32.135273]] - predict
110 -true
[[33.30315]] - predict
110 -true
[[49.970165]] - predict
110 -true
[[44.320965]] - predict
110 -true
[[54.58458]] - predict
110 -true
[[61.29937]] - predict
110 -true
[[69.30725]] - predict
110 -true
[[64.92128]] - predict
110 -true
[[83.81005]] - predict
110 -true
[[81.95427]] - predict
130 -true
[[110.2072]] - predict
130 -true
[[98.071]] - predict
130 -true
[[207.0335]] - predict
130 -true
[[165.37544]] - predict


In [11]:
df = pd.DataFrame({'y_predict':y_pred, 'y_true':y_true})
df.to_csv('C:/e/CRUPA/table2.csv')

In [12]:
res_df = pd.read_csv('C:/e/CRUPA/table1.csv', sep=',')
res_df['y_predict'] = res_df['y_predict'].apply(lambda x: round(float(x.strip('][')), 1))
res_df['error'] = round(res_df['y_predict'] - res_df['y_true'], 1)
res_df['error_in_%'] = round((res_df['y_predict'] - res_df['y_true']) * 100 / res_df['y_true'], 0)

In [13]:
res_df['error_in_%']

0       17.0
1       28.0
2       26.0
3     1004.0
4      100.0
5      -67.0
6       -6.0
7      -28.0
8       17.0
9       14.0
10      -4.0
11       5.0
12       0.0
13     -10.0
14     -11.0
15     -18.0
16       6.0
17     -25.0
18     -42.0
19     -23.0
20     -33.0
21      -9.0
22     -15.0
23     -32.0
24     -36.0
25     -31.0
26     -23.0
27     -32.0
28     -24.0
29     144.0
Name: error_in_%, dtype: float64

# MODEL 2

In [19]:
images_list = []
images_batch = []
mass_list = []
mass_batch = []
j = 0
img_shape = (224,224)
for i, file in enumerate(images_info_df['image_name']):
  #print(i,file)
  if j < 5:
    j += 1
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_shape)
    images_batch.append(img)
    mass_batch.append(images_info_df['mass'][i])
    
  else:
    j = 0 
    images_list.append(images_batch)
    mass_list.append(int(images_info_df['mass'][i])) 
    images_batch = [] 
    mass_batch = []

images_list = np.array(images_list)
mass_list = np.array(mass_list)    

print(mass_list)

[100 100 100 100 100 100 100 100 100 100  10  10  10  10  10  10  10  10
  10  10 110 110 110 110 110 110 110 110 110 110 130 130 130 130 130 130
 130 130 130 130 140 140 140 140 140 140 140 140 140 140 150 150 150 150
 150 150 150 150 150 150 160 160 160 160 160 160 160 160 160 160 170 170
 170 170 170 170 170 170 170 170 180 180 180 180 180 180 180 180 180 180
 190 190 190 190 190 190 190 190 190 190 200 200 200 200 200 200 200 200
 200 200  20  20  20  20  20  20  20  20  20  20  30  30  30  30  30  30
  30  30  30  30  40  40  40  40  40  40  40  40  40  40  50  50  50  50
  50  50  50  50  50  50  60  60  60  60  60  60  60  60  60  60  70  70
  70  70  70  70  70  70  70  70  80  80  80  80  80  80  80  80  80  80
  90  90  90  90  90  90  90  90  90  90 100 100  10  10 110 110 120 120
 130 130 140 140 150 150 160 160 170 170 180 180 190 190 200 200  20  30
  30  40  40  50  50  60  60  70  70  80  90  90]


In [14]:
import keras
from keras.layers import Dense, LSTM, \
    Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential
from keras.applications.vgg16 import VGG16

vgg = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

for layer in vgg.layers[:-4]:
    layer.trainable = False

model = Sequential()

model.add(
    TimeDistributed(vgg, input_shape=(5, 224, 224, 3))
)

model.add(
    TimeDistributed(
        Flatten()
    )
)
model.add(LSTM(256, activation='relu', return_sequences=False))

model.add(Dense(64, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(1, activation='linear'))
model.compile(optimizer = 'adam', loss='mean_absolute_error')

In [16]:
model = keras.models.load_model('C:/e/CRUPA/main_model/e35_val33.881947.h5')

In [25]:
y_pred = []
y_true = []
for i in range(0,100,5):
    
    img_sample = images_list[i]
    img_sample = np.expand_dims(img_sample, axis=0)
    pred = model.predict(img_sample)
    y_pred.append(pred)
    y_true.append(images_info_df['mass'][i])
    print(images_info_df['mass'][i], '-true')
    print(pred, '- predict')

100 -true
[[96.75077]] - predict
100 -true
[[90.840324]] - predict
100 -true
[[23.746256]] - predict
100 -true
[[20.972294]] - predict
100 -true
[[96.38449]] - predict
100 -true
[[114.46617]] - predict
100 -true
[[127.2355]] - predict
100 -true
[[131.53354]] - predict
100 -true
[[134.4623]] - predict
100 -true
[[125.623344]] - predict
100 -true
[[137.4307]] - predict
100 -true
[[140.13605]] - predict
10 -true
[[173.78113]] - predict
10 -true
[[157.32368]] - predict
10 -true
[[156.44904]] - predict
10 -true
[[157.05066]] - predict
10 -true
[[156.9496]] - predict
10 -true
[[170.66502]] - predict
10 -true
[[151.21518]] - predict
10 -true
[[181.04593]] - predict


In [26]:
df = pd.DataFrame({'y_predict':y_pred, 'y_true':y_true})
df.to_csv('C:/e/CRUPA/table2.csv')

In [27]:
res_df = pd.read_csv('C:/e/CRUPA/table1.csv', sep=',')
res_df['y_predict'] = res_df['y_predict'].apply(lambda x: round(float(x.strip('][')), 1))
res_df['error'] = round(res_df['y_predict'] - res_df['y_true'], 1)
res_df['error_in_%'] = round((res_df['y_predict'] - res_df['y_true']) * 100 / res_df['y_true'], 0)

In [28]:
res_df['error_in_%']

0       17.0
1       28.0
2       26.0
3     1004.0
4      100.0
5      -67.0
6       -6.0
7      -28.0
8       17.0
9       14.0
10      -4.0
11       5.0
12       0.0
13     -10.0
14     -11.0
15     -18.0
16       6.0
17     -25.0
18     -42.0
19     -23.0
20     -33.0
21      -9.0
22     -15.0
23     -32.0
24     -36.0
25     -31.0
26     -23.0
27     -32.0
28     -24.0
29     144.0
Name: error_in_%, dtype: float64